In [1]:
import json
import numpy as np
import music21 as m21
import tensorflow.keras as keras
# from ipynb.fs.full.preprocess import SEQUENCE_LENGTH, MAPPING_PATH
SEQUENCE_LENGTH = 64
MAPPING_PATH = "mapping.json"

In [20]:
class MelodyGenerator:
    def __init__(self, model_path="model.h5"):
        self.model_path = model_path
        self.model = keras.models.load_model(model_path)
        
        with open(MAPPING_PATH, "r") as fp:
            self._mappings = json.load(fp)
        
        self._start_symbols = ["/"] * SEQUENCE_LENGTH
        
    def generate_melody(self, seed, num_steps, max_sequence_length, temperature):
        # Seed is a piece of melody that we want to input into the network so it can generate from that; E.g. "64 _ 63 _ _ "
        # Num_steps how many we want the network to generate
        # Max_sequence_length tells us how many steps we want to consider into the network
        # Temperature tells how we sample output symbols
        
        # Create seed with start symbols
        seed = seed.split()
        melody = seed
        # First part of melody is already given
        seed = self._start_symbols + seed
        
        # Map seed to integers
        seed = [ self._mappings[symbol] for symbol in seed ]
        
        # We now have all we need to start generating symbols
        for _ in range(num_steps):
            
            # Limit the seed to max_sequence_length
            seed = seed[-max_sequence_length:]
            
            # One-hot encode the seed
            onehot_seed = keras.utils.to_categorical(seed, num_classes=len(self._mappings))
            # ( max_sequence_length, num of symbols in the vocabulary )
            
            # Prepare input for keras by adding extra dimension
            onehot_seed = onehot_seed[ np.newaxis, ... ] # Now ( 1, max_sequence_length, num of symbols in the vocabulary )
            
            # Make a prediction
            probabilities = self.model.predict(onehot_seed)[0]
            # [ 0.2, 0.4, 0.1, 0.3 ] -> 1
        
            output_int = self._sample_with_temperature(probabilities, temperature)
            
            # Update seed
            seed.append(output_int)
            
            # Map int to encoding
            output_symbol = [key for key, value in self._mappings.items() if value == output_int][0]
            
            # Check whether we are at the end of a melody
            if output_symbol == "/":
                break
                
            # Update the melody
            melody.append(output_symbol)
        
        return melody

            
    def _sample_with_temperature(self, probabilities, temperature):
        # temperature -> infinity => all probs tend to get remodeled and values tend to become the same
        # temperature -> 0 => probs remodeled so that highest value becomes 1
        # temperature -> 1 => probs do not change
        # Decides how explorative you want to be. Closer to 0 => more deterministic; -> infinity => more unpredictable
        predictions = np.log(probabilities) / temperature
        probabilities = np.exp(predictions) / np.sum(np.exp(predictions))
        
        choices = range(len(probabilities)) # [0, 1, 2, 3]
        index = np.random.choice(choices, p=probabilities)
        
        return index
    
    
    def save_melody(self, melody, step_duration=0.25, format="midi", file_name="mel.midi"):
        
        # Create a music21 stream
        stream = m21.stream.Stream()
        
        # Parse all the symbols in the melody and create note/rest objects
        # 60 _ _ _ r _ 62 _
        start_symbol = None
        step_counter = 1
        
        for i, symbol in enumerate(melody):
            
            # Handle case in which we have a note/rest
            if symbol != "_" or i + 1 == len(melody): # To also consider case where last symbol is _
                
                # Ensure we are dealing with note/rest beyond the first one
                if start_symbol is not None:
                    quarter_length_duration = step_duration * step_counter
                    
                    # Handle rest
                    if start_symbol == "r":
                        m21_event = m21.note.Rest(quarterLength=quarter_length_duration)
                    
                    # Handle note
                    else:
                        m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)
                    
                    stream.append(m21_event)
                    
                    # Reset step counter
                    step_counter = 1
                    
                start_symbol = symbol
            
            # Handle case in which we have a prolongation sign "_"
            else:
                step_counter += 1
        
        # Write the m21 stream to a midi file
        stream.write(format, file_name)
        
        

In [21]:
mg = MelodyGenerator()

# I just picked a random piece from one of the files in the dataset folder
seed = "67 _ 67 _ 67 _ _ 65 64 _ 64 _ 64 _ _"
seed2 = "67 _ _ _ _ _ 65 _ 64 _ 62 _ 60 _ _ _"

melody = mg.generate_melody(seed, 500, SEQUENCE_LENGTH, 0.3)
print(melody)

['67', '_', '67', '_', '67', '_', '_', '65', '64', '_', '64', '_', '64', '_', '_', '_', '67', '_', '_', '_', '67', '_', '67', '_', '64', '_', '_', '_', '67', '_', '_', '_', '67', '_', '_', '_', '65', '_', '_', '_', '64', '_', '_', '_', 'r', '_', '_', '_', '60', '_', '64', '_', '67', '_', '_', '_', '64', '_', '_', '_', '69', '_', '67', '_', '65', '_', '64', '_', '62', '_', '_', '_', '62', '_', '_', '_', '60', '_', '_', '_', 'r', '_', '_', '_']


In [22]:
mg.save_melody(melody)